In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle, gzip, math, torch, matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor

In [3]:
MNIST_URL = 'http://deeplearning.net/data/mnist/mnist.pkl'

In [4]:
def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid,y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, ((x_train,y_train,x_valid, y_valid)))
               
def normalize(x, m, s): return (x-m)/s        

In [5]:
x_train, y_train, x_valid, y_valid = get_data()

In [6]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.1304), tensor(0.3073))

In [7]:
x_train  = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [8]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.0001), tensor(1.))

In [9]:
def test_near_zero(a, tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [10]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [11]:
n, m = x_train.shape
c = y_train.max()+1
n, m, c

(50000, 784, tensor(10))

In [12]:
nh = 50

In [13]:
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [14]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [15]:
x_valid.mean(), x_valid.std()

(tensor(-0.0057), tensor(0.9924))

In [16]:
def lin(x, w, b): return x@w + b 

In [17]:
t = lin(x_valid, w1, b1)

In [18]:
t.mean(), t.std()

(tensor(0.0923), tensor(0.9887))

In [19]:
def relu(x): return x.clamp_min(0.)

In [20]:
t = relu(lin(x_valid, w1, b1))

In [21]:
t.mean(), t.std()

(tensor(0.4393), tensor(0.6018))

In [22]:
w1 = torch.randn(m,nh)*math.sqrt(2/m)

In [23]:
w1.mean(), w1.std()

(tensor(-0.0001), tensor(0.0507))

In [24]:
t = relu(lin(x_train, w1, b1))

In [25]:
t.mean(), t.std()

(tensor(0.5234), tensor(0.7782))

In [26]:
from torch.nn import init

In [27]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out')
t=relu(lin(x_train, w1, b1))

In [28]:
t.mean(), t.std()

(tensor(0.4423), tensor(0.7298))

In [29]:
init.kaiming_normal_??

In [30]:
w1.mean(), w1.std()

(tensor(-8.8014e-06), tensor(0.0506))

In [31]:
t.mean(), t.std()

(tensor(0.4423), tensor(0.7298))

In [32]:
w1.shape

torch.Size([784, 50])

In [33]:
def relu(x): return x.clamp_min(0.) - 0.5

In [34]:
w1 = torch.randn(m,nh)*math.sqrt(2./m)
t1 = relu(lin(x_train, w1, b1))
t1.mean(), t1.std()

(tensor(0.0157), tensor(0.7933))

In [35]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    
    return l3

In [36]:
%timeit -n 10 _=model(x_valid)

15.9 ms ± 460 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [37]:
assert model(x_valid).shape == torch.Size([x_valid.shape[0],1])

In [38]:
model(x_valid).shape

torch.Size([10000, 1])

In [39]:
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [40]:
y_train, y_valid = y_train.float(), y_valid.float()

In [41]:
preds = model(x_train)

In [42]:
preds.shape

torch.Size([50000, 1])

In [43]:
mse(preds, y_train)

tensor(25.3803)

In [44]:
def mse_grad(inp, targ):
    
    inp.g = 2 * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [45]:
def relu_grad(inp, out):
    
    inp.g = (inp>0).float() * out.g

In [46]:
def lin_grad(inp, out, w, b):
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [47]:
def forward_and_backward(inp, targ):
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    
    loss = mse(out, targ)
    
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [48]:
forward_and_backward(x_train, y_train)

In [49]:
w1g = w1.g.clone()
b1g = b1.g.clone()
w2g = w2.g.clone()
b2g = b2.g.clone()
ig = x_train.g.clone()

In [50]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [51]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [52]:
loss = forward(xt2, y_train)

In [53]:
loss.backward()

In [54]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [55]:
class Lin():
    def __init__(self, w, b): self.w, self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [56]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [57]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()
        

In [58]:
w1.g, b1.g, w2.g, b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [59]:
%time loss = model(x_train, y_train)

CPU times: user 88 ms, sys: 0 ns, total: 88 ms
Wall time: 88.6 ms


In [60]:
%time model.backward()

CPU times: user 3.2 s, sys: 3.34 s, total: 6.55 s
Wall time: 6.55 s


In [61]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [62]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [63]:
class Lin(Module):
    def __init__(self, w, b): self.w, self.b = w, b
        
    def forward(self, inp): return inp@self.w + self.b 
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [64]:
class Mse(Module):
    def forward(self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) /targ.shape[0]

In [65]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [66]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [67]:
%time loss = model(x_train, y_train)

CPU times: user 88 ms, sys: 0 ns, total: 88 ms
Wall time: 87.7 ms


In [68]:
%time model.backward()

CPU times: user 168 ms, sys: 72 ms, total: 240 ms
Wall time: 244 ms


In [69]:
class Lin(Module):
    def __init__(self,w,b): self.w, self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g@self.w.t()
        self.w.g = inp.t()@out.g
        self.b.g = out.g.sum(0)
        

In [70]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [71]:
%time loss = model(x_train, y_train)

CPU times: user 88 ms, sys: 4 ms, total: 92 ms
Wall time: 90.1 ms


In [72]:
%time model.backward()

CPU times: user 160 ms, sys: 76 ms, total: 236 ms
Wall time: 236 ms


In [73]:
from torch import nn

In [74]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in, nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [75]:
model = Model(m, nh, 1)

In [76]:
%time loss = model(x_train, y_train)

CPU times: user 92 ms, sys: 0 ns, total: 92 ms
Wall time: 95.7 ms


In [77]:
%time loss.backward()

CPU times: user 68 ms, sys: 0 ns, total: 68 ms
Wall time: 69.5 ms
